In [1]:
# Import Dependencies
import pandas as pd 
import numpy
import requests
import json
import datetime as dt
from sqlalchemy import create_engine

# From open data to retrieve lat and lon of station_id 

In [2]:
url = "https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_information"
print(requests.get(url))

<Response [200]>


In [3]:
# Pretty Print the output of the JSON
response = requests.get(url).json()
#print(json.dumps(response, indent=4, sort_keys=True))
station_id = response['data']['stations'][0]['station_id']
lat = response['data']['stations'][0]['lat']
lon = response['data']['stations'][0]['lon']

In [4]:
# Retrieve data: station_id, lat, lon 
stations = response['data']['stations']
lat = response['data']['stations']
lon = response['data']['stations']

station_id = []
station_name =[]
lats = []
lons = []
for station in stations: 
    #print(station["station_id"])
    station_id.append(station["station_id"])
    station_name.append(station["name"])
    lats.append(station['lat'])
    lons.append(station['lon'])

In [6]:
# Create data frame 
station_coordinate = pd.DataFrame({"station_id":station_id,"station_name":station_name,"lat":lats,"lon":lons}).copy()
station_coordinate.head()

,station_id,station_name,lat,lon
0,7000,Fort York Blvd / Capreol Ct,43.639832,-79.395954
1,7001,Lower Jarvis St / The Esplanade,43.647992,-79.370907
2,7002,St. George St / Bloor St W,43.667333,-79.399429
3,7003,Madison Ave / Bloor St W,43.667158,-79.402761
4,7004,University Ave / Elm St,43.656518,-79.389099


In [7]:
# Create data frame 
stations = pd.DataFrame({"station_id":station_id,"station_name":station_name})
#stations = stations.set_index(['station_id'])
stations.head()

,station_id,station_name
0,7000,Fort York Blvd / Capreol Ct
1,7001,Lower Jarvis St / The Esplanade
2,7002,St. George St / Bloor St W
3,7003,Madison Ave / Bloor St W
4,7004,University Ave / Elm St


# Import and clean the quarterly bikeshare files

In [8]:
quarter_one_file_path = "../Project2/2017 Data/Bikeshare Ridership (2017 Q1).csv"
quarter_one = pd.read_csv(quarter_one_file_path)
quarter_one.head()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_id,from_station_name,to_station_id,to_station_name,user_type
0,712382,1/1/2017 0:00,1/1/2017 0:03,223,7051,Wellesley St E / Yonge St Green P,7089,Church St / Wood St,Member
1,712383,1/1/2017 0:00,1/1/2017 0:05,279,7143,Kendal Ave / Bernard Ave,7154,Bathurst Subway Station,Member
2,712384,1/1/2017 0:05,1/1/2017 0:29,1394,7113,Parliament St / Aberdeen Ave,7199,College St W / Markham St,Member
3,712385,1/1/2017 0:07,1/1/2017 0:21,826,7077,College Park South,7010,King St W / Spadina Ave,Member
4,712386,1/1/2017 0:08,1/1/2017 0:12,279,7079,McGill St / Church St,7047,University Ave / Gerrard St W,Member


In [9]:
quarter_two_file_path = "../Project2/2017 Data/Bikeshare Ridership (2017 Q2).csv"
quarter_two = pd.read_csv(quarter_two_file_path)
quarter_two.head()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_id,from_station_name,to_station_id,to_station_name,user_type
0,868665,1/4/2017 0:01,1/4/2017 0:06,297,7073,Bremner Blvd / Spadina Ave,7211,Fort York Blvd / Garrison Rd,Member
1,868666,1/4/2017 0:07,1/4/2017 0:11,236,7038,Dundas St / Yonge St,7163,Yonge St / Wood St,Member
2,868667,1/4/2017 0:08,1/4/2017 0:14,380,7012,Elizabeth St / Edward St (Bus Terminal),7069,Queen St W / Spadina Ave,Member
3,868668,1/4/2017 0:12,1/4/2017 0:34,1299,7172,Strachan Ave / Princes' Blvd,7069,Queen St W / Spadina Ave,Member
4,868669,1/4/2017 0:13,1/4/2017 0:18,323,7039,Simcoe St / Dundas St W,7021,Bay St / Albert St,Member


In [10]:
# Merge quarter_one and quarter_two
first_half_year = pd.concat([quarter_one,quarter_two])

# Change to datetime 
first_half_year['trip_start_time'] = pd.to_datetime(first_half_year['trip_start_time'],format = '%d/%m/%Y %H:%M')
first_half_year['trip_stop_time'] = pd.to_datetime(first_half_year['trip_stop_time'],format = '%d/%m/%Y %H:%M')
first_half_year.dropna()
first_half_year.head()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_id,from_station_name,to_station_id,to_station_name,user_type
0,712382,2017-01-01 00:00:00,2017-01-01 00:03:00,223,7051,Wellesley St E / Yonge St Green P,7089,Church St / Wood St,Member
1,712383,2017-01-01 00:00:00,2017-01-01 00:05:00,279,7143,Kendal Ave / Bernard Ave,7154,Bathurst Subway Station,Member
2,712384,2017-01-01 00:05:00,2017-01-01 00:29:00,1394,7113,Parliament St / Aberdeen Ave,7199,College St W / Markham St,Member
3,712385,2017-01-01 00:07:00,2017-01-01 00:21:00,826,7077,College Park South,7010,King St W / Spadina Ave,Member
4,712386,2017-01-01 00:08:00,2017-01-01 00:12:00,279,7079,McGill St / Church St,7047,University Ave / Gerrard St W,Member


In [11]:
quarter_three_file_path = "../Project2/2017 Data/Bikeshare Ridership (2017 Q3).csv"
quarter_three = pd.read_csv(quarter_three_file_path)

#Change trip_start_time and trip_stop_time to datetime 
quarter_three['trip_start_time'] = pd.to_datetime(quarter_three['trip_start_time'],format = '%m/%d/%Y %H:%M')
quarter_three['trip_stop_time'] = pd.to_datetime(quarter_three['trip_stop_time'],format = '%m/%d/%Y %H:%M')
quarter_three.head()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_name,to_station_name,user_type
0,1253914,2017-07-01 00:00:00,2017-07-01 00:15:00,910,Princess St / Adelaide St E,424 Wellington St W,Member
1,1253915,2017-07-01 00:01:00,2017-07-01 00:15:00,837,Fort York Blvd / Capreol Crt,HTO Park (Queens Quay W),Casual
2,1253916,2017-07-01 00:01:00,2017-07-01 00:14:00,786,Fort York Blvd / Capreol Crt,HTO Park (Queens Quay W),Casual
3,1253917,2017-07-01 00:01:00,2017-07-01 00:25:00,1420,Elizabeth St / Edward St (Bus Terminal),Boston Ave / Queen St E,Casual
4,1253918,2017-07-01 00:01:00,2017-07-01 00:25:00,1437,Elizabeth St / Edward St (Bus Terminal),Boston Ave / Queen St E,Casual


In [12]:
quarter_four_file_path = "../Project2/2017 Data/Bikeshare Ridership (2017 Q4).csv"
quarter_four= pd.read_csv(quarter_four_file_path)
quarter_four = quarter_four.dropna()

#Change trip_start_time and trip_stop_time to datetime format 
quarter_four['trip_start_time'] = pd.to_datetime(quarter_four['trip_start_time'],format = '%m/%d/%y %H:%M:%S')
quarter_four['trip_stop_time'] = pd.to_datetime(quarter_four['trip_stop_time'],format = '%m/%d/%y %H:%M:%S')
quarter_four.head()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_name,to_station_name,user_type
0,1971685,2017-10-01 00:00:01,2017-10-01 00:14:10,849,Queen St W / James St,Taddle Creek Park,Member
1,1971686,2017-10-01 00:00:01,2017-10-01 00:00:27,26,Stewart St / Bathurst St - SMART,Stewart St / Bathurst St - SMART,Casual
2,1971687,2017-10-01 00:00:22,2017-10-01 00:22:12,1310,Hayter St / Laplante Ave,Queen St W / Portland St,Member
3,1971688,2017-10-01 00:00:38,2017-10-01 00:11:09,631,Beverly St / College St,Beverly St / Dundas St W,Member
4,1971689,2017-10-01 00:00:55,2017-10-01 00:20:53,1198,Thompson St / Broadview Ave - SMART,Thompson St / Broadview Ave - SMART,Member


In [13]:
# Merge quarter_three and quarter_four
second_half_year = pd.concat([quarter_three,quarter_four])
second_half_year.head()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_name,to_station_name,user_type
0,1253914,2017-07-01 00:00:00,2017-07-01 00:15:00,910,Princess St / Adelaide St E,424 Wellington St W,Member
1,1253915,2017-07-01 00:01:00,2017-07-01 00:15:00,837,Fort York Blvd / Capreol Crt,HTO Park (Queens Quay W),Casual
2,1253916,2017-07-01 00:01:00,2017-07-01 00:14:00,786,Fort York Blvd / Capreol Crt,HTO Park (Queens Quay W),Casual
3,1253917,2017-07-01 00:01:00,2017-07-01 00:25:00,1420,Elizabeth St / Edward St (Bus Terminal),Boston Ave / Queen St E,Casual
4,1253918,2017-07-01 00:01:00,2017-07-01 00:25:00,1437,Elizabeth St / Edward St (Bus Terminal),Boston Ave / Queen St E,Casual


In [14]:
# Insert station id to the second_half_year
add_station_ids = second_half_year.merge(stations, left_on = "from_station_name", right_on ="station_name", how = "left")\
   .merge(stations, left_on = "to_station_name", right_on ="station_name", how = "left")

add_station_ids.head()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_name,to_station_name,user_type,station_id_x,station_name_x,station_id_y,station_name_y
0,1253914,2017-07-01 00:00:00,2017-07-01 00:15:00,910,Princess St / Adelaide St E,424 Wellington St W,Member,7060,Princess St / Adelaide St E,7123,424 Wellington St W
1,1253915,2017-07-01 00:01:00,2017-07-01 00:15:00,837,Fort York Blvd / Capreol Crt,HTO Park (Queens Quay W),Casual,NaN,NaN,7175,HTO Park (Queens Quay W)
2,1253916,2017-07-01 00:01:00,2017-07-01 00:14:00,786,Fort York Blvd / Capreol Crt,HTO Park (Queens Quay W),Casual,NaN,NaN,7175,HTO Park (Queens Quay W)
3,1253917,2017-07-01 00:01:00,2017-07-01 00:25:00,1420,Elizabeth St / Edward St (Bus Terminal),Boston Ave / Queen St E,Casual,7012,Elizabeth St / Edward St (Bus Terminal),7096,Boston Ave / Queen St E
4,1253918,2017-07-01 00:01:00,2017-07-01 00:25:00,1437,Elizabeth St / Edward St (Bus Terminal),Boston Ave / Queen St E,Casual,7012,Elizabeth St / Edward St (Bus Terminal),7096,Boston Ave / Queen St E


In [15]:
second_half_year = add_station_ids.copy()

#Rename columns name
second_half_year = second_half_year.rename(columns={"station_id_x": "from_station_id",
                                                    "station_id_y": "to_station_id"})

In [16]:
#Reposition columns
second_half_year = second_half_year[['trip_id',
                                     'trip_start_time', 
                                     'trip_stop_time',
                                     'trip_duration_seconds',
                                     'from_station_id',
                                     'from_station_name',
                                     'to_station_id', 
                                     'to_station_name', 
                                     'user_type']]

second_half_year = second_half_year.dropna()
second_half_year.head()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_id,from_station_name,to_station_id,to_station_name,user_type
0,1253914,2017-07-01 00:00:00,2017-07-01 00:15:00,910,7060,Princess St / Adelaide St E,7123,424 Wellington St W,Member
3,1253917,2017-07-01 00:01:00,2017-07-01 00:25:00,1420,7012,Elizabeth St / Edward St (Bus Terminal),7096,Boston Ave / Queen St E,Casual
4,1253918,2017-07-01 00:01:00,2017-07-01 00:25:00,1437,7012,Elizabeth St / Edward St (Bus Terminal),7096,Boston Ave / Queen St E,Casual
5,1253919,2017-07-01 00:03:00,2017-07-01 00:25:00,1332,7026,Bay St / St. Joseph St,7046,Niagara St / Richmond St W,Member
6,1253920,2017-07-01 00:04:00,2017-07-01 00:53:00,2930,7006,Bay St / College St (East Side),7038,Dundas St W / Yonge St,Casual


In [17]:
# Merge quarters to a year 
bike_share_2017 =  pd.concat([first_half_year ,second_half_year])
bike_share_2017.head()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_id,from_station_name,to_station_id,to_station_name,user_type
0,712382,2017-01-01 00:00:00,2017-01-01 00:03:00,223,7051,Wellesley St E / Yonge St Green P,7089,Church St / Wood St,Member
1,712383,2017-01-01 00:00:00,2017-01-01 00:05:00,279,7143,Kendal Ave / Bernard Ave,7154,Bathurst Subway Station,Member
2,712384,2017-01-01 00:05:00,2017-01-01 00:29:00,1394,7113,Parliament St / Aberdeen Ave,7199,College St W / Markham St,Member
3,712385,2017-01-01 00:07:00,2017-01-01 00:21:00,826,7077,College Park South,7010,King St W / Spadina Ave,Member
4,712386,2017-01-01 00:08:00,2017-01-01 00:12:00,279,7079,McGill St / Church St,7047,University Ave / Gerrard St W,Member


In [18]:
bike_share_2017.isnull().sum()

trip_id                  0
trip_start_time          0
trip_stop_time           0
trip_duration_seconds    0
from_station_id          0
from_station_name        0
to_station_id            0
to_station_name          0
user_type                0
dtype: int64

# Load data to database

In [19]:
connection_string = "postgres:postgres@localhost:5432/bikeshare_db"
engine = create_engine(f'postgresql://{connection_string}')

In [20]:
# Confirm tables
engine.table_names()

['station_coordinate', 'bike_share_2017']

In [35]:
station_coordinate.to_sql(name='station_coordinate', con=engine, if_exists='replace', index=False)

In [44]:
#bike_share_2017[bike_share_2017['from_station_id']==7094]

In [ ]:
bike_share_2017.to_sql(name='bike_share_2017', con=engine, if_exists='replace')